문제 2 10점  
문제.  
사용자들에게 1점도 받고 5점도 받은 적이 있는 영화의 제목의 길이 중 가장 긴 영화제목의 길이?

동일 영화가 어떤 사용자에게는 1점, 어떤 사용자에게는 5점을 받은 경우를 말함  
힌트: 어떤 column의 길이의 max 를 select할 때 max 함수와 length함수를 사용.  
예를 들어 uitem\["title"]의 길이의 max는 select(max(length(uitem\["title"]))) 로 계산 할 수 있음  
제한조건: 마지막 action으로 답이 출력되도록 코드를 작성하시오. 즉, 최종 답을 계산하기 위해 pyspark action이 아닌 python 코드를 사용하지 마시오.  

In [1]:
import findspark
findspark.init()

# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my app").master("local").getOrCreate() 

# get context from the session
sc = spark.sparkContext

In [2]:
from pyspark.sql.types import *
schema = StructType([
    StructField("uid", LongType(), False),
    StructField("mid", LongType(), False),
    StructField("rate", LongType(), False),
    StructField("timestamp", LongType(), False)
])
udata = spark.read.format("csv").option("sep", "\t" ).schema(schema).load("u.data")
print(udata.show(3))
udata.printSchema()

+---+---+----+---------+
|uid|mid|rate|timestamp|
+---+---+----+---------+
|196|242|   3|881250949|
|186|302|   3|891717742|
| 22|377|   1|878887116|
+---+---+----+---------+
only showing top 3 rows

None
root
 |-- uid: long (nullable = true)
 |-- mid: long (nullable = true)
 |-- rate: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [3]:
schema = StructType([
    StructField("mid", LongType(), False),
    StructField("title", StringType(), False)
])
uitem = spark.read.format("csv").option("sep", "|").schema(schema).load("u.item")
print(uitem.show(3))
uitem.printSchema() 

+---+-----------------+
|mid|            title|
+---+-----------------+
|  1| Toy Story (1995)|
|  2| GoldenEye (1995)|
|  3|Four Rooms (1995)|
+---+-----------------+
only showing top 3 rows

None
root
 |-- mid: long (nullable = true)
 |-- title: string (nullable = true)



In [4]:
from pyspark.sql.functions import *
uitem.select("mid", "title")\
.join(udata.select("mid", "rate"), uitem["mid"] == udata["mid"], "inner")\
.select("title", "rate")\
.filter( col('rate') == (1 & 5) )\
.select( max( length( col( "title") ) )).show() 


+------------------+
|max(length(title))|
+------------------+
|                81|
+------------------+

